In [2]:
library(regioneR)
library(rtracklayer)
library(BSgenome.Hsapiens.UCSC.hg19)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


At

# Import Data

## Mappability bed file

The mappability bed file specifies the regions where the sequencing strategy can detect cleavage events.

In [3]:
map_bed <- toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/mappability.stranded.hg38.bed',
                                sep='\t'))

In [8]:
map_bed_chr19 <- map_bed[map_bed@seqnames == 'chr19']

## Consensus

In [5]:
consensus_bed <- toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/consensus_footprints_and_collapsed_motifs_hg38.bed',
                                sep='\t'))

In [9]:
consensus_bed_chr19 <- consensus_bed[consensus_bed@seqnames == 'chr19']

## CD20 Expected Cleavages

In [16]:
cd20_beds <- list(cd20_0.0001 = toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/interval.all.fps.0.0001.bed', sep='\t')),
                  cd20_0.001 = toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/interval.all.fps.0.001.bed',sep='\t')),
                  cd20_0.01 = toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/interval.all.fps.0.01.bed',sep='\t')),
                  cd20_0.05 = toGRanges(read.table('/home/jupyter/human-TF-footprinting/Data/interval.all.fps.0.05.bed',sep='\t')))

In [10]:
# Subset to Chr19
for (bed_df in names(cd20_beds)) {
    cd20_beds[[bed_df]] <- cd20_beds[[bed_df]][cd20_beds[[bed_df]]@seqnames == "chr19"]
}

## Interval.all.bedgraph

Per-nucleotide footprint statistics

In [13]:
cd20_all <- toGRanges('/home/jupyter/human-TF-footprinting/Data/interval.all.bedgraph')

In [15]:
cd20_all_chr19 <- cd20_all[cd20_all@seqnames == 'chr19']

# Consensus Analysis

In [17]:
consensus.analysis <- function(bed_list,
                               map_bed,
                               consensus_bed){
    
    print(paste('Number of regions in mappable regions: ', nrow(toDataframe(map_bed)), 
                sep=' '))
    
    cat('\n')

    print(paste('Number of regions in the consensus: ', nrow(toDataframe(consensus_bed)), 
                sep=' '))
    
    cat('\n')

    
    # number of regions in dataset of interest
    for (bed_df in names(bed_list)) {
        print(paste(bed_df, 'number of regions:', nrow(toDataframe(bed_list[[bed_df]])), sep=' '))
    }
    
    cat('\n')
    
    # check if the dataset of interest is a proper subset of mappable regions
        for (bed_df in names(bed_list)) {
            print(bed_df)
            print(paste('Number of overlaps in mappable reions:',
                        numOverlaps(bed_list[[bed_df]], 
                                    map_bed, 
                                    count.once=TRUE), sep=' '))
            print(paste('Number of overlaps in consensus:', 
                        numOverlaps(bed_list[[bed_df]], 
                                    consensus_bed, 
                                    count.once=TRUE), sep=' '))
            cat('\n')

        }
    
}

consensus.analysis(cd20_beds, map_bed, consensus_bed)
# consensus.analysis(cd20_beds, map_bed_chr19, consensus_bed_chr19)

[1] "Number of regions in mappable regions:  64470670"

[1] "Number of regions in the consensus:  4465728"

[1] "cd20_0.0001 number of regions: 116426"
[1] "cd20_0.001 number of regions: 192149"
[1] "cd20_0.01 number of regions: 414089"
[1] "cd20_0.05 number of regions: 795635"

[1] "cd20_0.0001"
[1] "Number of overlaps in mappable reions: 114172"
[1] "Number of overlaps in consensus: 109073"

[1] "cd20_0.001"
[1] "Number of overlaps in mappable reions: 188635"
[1] "Number of overlaps in consensus: 168529"

[1] "cd20_0.01"
[1] "Number of overlaps in mappable reions: 407098"
[1] "Number of overlaps in consensus: 308905"

[1] "cd20_0.05"
[1] "Number of overlaps in mappable reions: 783570"
[1] "Number of overlaps in consensus: 494636"



In [7]:
set.seed(10)

In [21]:
genomic_mask <- getGenomeAndMask('hg19', getGenomeAndMask('hg19', mask=map_bed))

In [24]:
hg19_bed <- getGenome('hg19')
genomic_mask <- subtractRegions(hg19_bed, map_bed)

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 1 out-of-bound range located on sequence chr17.
  Note that ranges located on a sequence whose length is unknown (NA) or
  on a circular sequence are not considered out-of-bound (use
  seqlengths() and isCircular() to get the lengths and circularity flags
  of the underlying sequences). You can use trim() to trim these ranges.
  See ?`trim,GenomicRanges-method` for more information.”
Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 1 out-of-bound range located on sequence chr17.
  Note that ranges located on a sequence whose length is unknown (NA) or
  on a circular sequence are not considered out-of-bound (use
  seqlengths() and isCircular() to get the lengths and circularity flags
  of the underlying sequences). You can use trim() to trim these ranges.
  See ?`trim,GenomicRanges-method` for more information.”
Warning message in valid.Genomic

In [ ]:
map_randomized <- createRandomRegions(nregions=length(map_bed),
                                      length.mean=mean(map_bed@ranges@width),
                                      length.sd=sqrt(var(map_bed@ranges@width)),
                                      genome='hg19',
                                      mask=genomic_mask)

In [1]:
map_randomized

ERROR: Error in eval(expr, envir, enclos): object 'map_randomized' not found
